In [3]:
from scr.tushare_api import TuShare
import pandas as pd

my_ts = TuShare()

,trade_date,ts_code,exalter,buy,buy_rate,sell,sell_rate,net_buy,side,reason
0,20220621,000017.SZ,华鑫证券有限责任公司杭州飞云江路证券营业部,20254270.0,3.86,319864.00,0.06,19934406.00,0,振幅值达15%的证券
1,20220621,000017.SZ,申港证券股份有限公司北京分公司,16640244.0,3.17,0.00,0.00,16640244.00,0,振幅值达15%的证券
2,20220621,000017.SZ,东莞证券股份有限公司浙江分公司,8795332.0,1.68,0.00,0.00,8795332.00,0,振幅值达15%的证券
3,20220621,000017.SZ,东方财富证券股份有限公司拉萨东环路第二证券营业部,6024128.0,1.15,3642782.16,0.69,2381345.84,0,振幅值达15%的证券
4,20220621,000017.SZ,东方财富证券股份有限公司拉萨东环路第一证券营业部,5804693.0,1.11,3517799.00,0.67,2286894.00,0,振幅值达15%的证券
...,...,...,...,...,...,...,...,...,...,...
815,20220621,900955.SH,华泰证券股份有限公司西安文艺北路证券营业部,NaN,NaN,4108.80,17.78,-4108.80,1,退市整理期
816,20220621,900955.SH,招商证券股份有限公司武汉中北路证券营业部,NaN,NaN,2024.00,8.76,-2024.00,1,退市整理期
817,20220621,900955.SH,中国中金财富证券有限公司北京宋庄路证券营业部,NaN,NaN,1955.00,8.46,-1955.00,1,退市整理期
818,20220621,900955.SH,中天证券股份有限公司上海延安西路证券营业部,NaN,NaN,1313.30,5.68,-1313.30,1,退市整理期


In [2]:
get_trade_days('2022-01-01','2022-01-31')

DatetimeIndex(['2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',
               '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13',
               '2022-01-14', '2022-01-17', '2022-01-18', '2022-01-19',
               '2022-01-20', '2022-01-21', '2022-01-24', '2022-01-25',
               '2022-01-26', '2022-01-27', '2022-01-28'],
              dtype='datetime64[ns]', freq=None)